In [ ]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
import time
import re
import openpyxl
import pandas as pd
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

In [ ]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder) # you can predefine where you wanna store things in case its needed
    profile.set_preference("browser.download.folderList", 2) # 0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
    profile.set_preference("browser.download.manager.showWhenStarting", False) #I dont wanna see a pop up for each download so i swith it off
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")

    # profile.install_addon('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse_2025/booking/booking/ublock_origin-1.55.0.xpi')
    # profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True) #dont want the pdf viewer to open

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,donwload=True):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

In [ ]:
# lets open booking:

dfolder='./downloads'
geko_path=''
link='https://www.booking.com/index.en.html'


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)


In [ ]:
browser.find_element(by=By.XPATH,value='//*[@id=":rh:"]').click()
# Input the place: Send your desired destination and combine with the element above
place = input('Where do you want to go?')
search1 = browser.find_element(by='xpath',value='//*[@id=":rh:"]')
search1.send_keys(place)

In [ ]:
# Now let's input the dates.
css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'
browser.find_element('css selector',css).click()

# Define a navigating function
def navigate_to_month(browser, current_month, current_year, target_month, target_year):
    months = ['January', 'February', 'March', 'April', 'May', 'June',
              'July', 'August', 'September', 'October', 'November', 'December']

    # Convert month names to numbers (1-12)
    current_month_num = months.index(current_month) + 1
    target_month_num = months.index(target_month) + 1

    # Calculate how many months we need to move forward
    months_diff = (target_year - current_year) * 12 + (target_month_num - current_month_num)

    # Click the next month button the calculated number of times
    for _ in range(months_diff):
        next_month_button = browser.find_element('css selector', 'button[aria-label="Next month"]')
        browser.execute_script("arguments[0].click();", next_month_button)
        time.sleep(1)

# Navigate to the time period we need
navigate_to_month(browser, "January", 2025, "May", 2025)


In [ ]:
# Input the date we need
from_day = input('Day from which you want to look for (In May)')
to_day = input('Day until which you want to look for (In June)')


for date in dates:
    if date.get_attribute("data-date") == f"2025-05-{from_day}":
        browser.execute_script("arguments[0].click();", date)
    if date.get_attribute("data-date") == f"2025-05-{to_day}":
        browser.execute_script("arguments[0].click();", date)
        break

# Looping across the calendar qould require additional code.

In [ ]:
# Search the hotels
my_xpath='/html/body/div[3]/div[2]/div/form/div/div[4]/button/span'

check_obscures(browser,my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')

In [ ]:
# Define the function to scroll down and load more results
def scroll_and_load_all(browser):
    while True:
        last_height = browser.execute_script("return document.body.scrollHeight")
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = browser.execute_script("return document.body.scrollHeight")

        # Double-check height in case content loads slowly
        if new_height == last_height:
            time.sleep(2)
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                pass
        else:
            last_height = new_height

        try:
            load_more_button = WebDriverWait(browser, 5).until(
                EC.element_to_be_clickable((By.XPATH,
                   "//button[@class='a83ed08757 c21c56c305 bf0537ecb5 f671049264 af7297d90d c0e0affd09']"
                   "//span[@class='e4adce92df' and contains(text(),'Load more results')]"))
            )
            browser.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
            time.sleep(1)
            load_more_button.click()
            time.sleep(3)
            WebDriverWait(browser, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, "//div[@class='f6431b446c a15b38c233']"))
            )
        except Exception as e:
            print(f"No more results to load: {e}")
            break

# After your search results are displayed:
scroll_and_load_all(browser)

In [ ]:
# First locate all hotel containers
hotel_containers = browser.find_elements('xpath', '//div[@data-testid="property-card-container"]')

# Lists to store data
hotels_data = []

for container in hotel_containers:
    try:
        # Initialize a dictionary for each hotel
        hotel_info = {}

        # Extract hotel name
        try:
            hotel_name = container.find_element('xpath', './/div[contains(@class, "f6431b446c")]').text
            hotel_info['name'] = hotel_name
        except:
            hotel_info['name'] = None

        # Extract hotel ID from URL and store the link
        try:
            link = container.find_element('xpath', './/a[contains(@class, "a78ca197d0")]').get_attribute('href')
            hotel_info['link'] = link  # Store the full link
            id_match = re.findall(r'matching_block_id=(\d+)_', link)
            hotel_info['id'] = id_match[0] if id_match else None
        except:
            hotel_info['id'] = None
            hotel_info['link'] = None

        # Extract rating
        try:
            rating_element = container.find_element('xpath', './/div[@class="a3b8729ab1 d86cee9b25"]')
            rating_match = re.findall(r'\d+\.\d+', rating_element.text)
            hotel_info['rating'] = float(rating_match[0]) if rating_match else None
        except:
            hotel_info['rating'] = None

        # Extract price
        try:
            price_element = container.find_element('xpath', './/span[@data-testid="price-and-discounted-price"]')
            price_text = price_element.text
            # Extract numbers from the price text
            price_match = re.findall(r'\d+,?\d*', price_text)
            if price_match:
                # Remove commas and convert to float
                price_value = float(price_match[0].replace(',', ''))
                hotel_info['price'] = price_value
            else:
                hotel_info['price'] = None
        except:
            hotel_info['price'] = None

        hotels_data.append(hotel_info)

    except Exception as e:
        print(f"Error processing a hotel: {str(e)}")
        continue

# Create DataFrame
df = pd.DataFrame(hotels_data)

In [ ]:
# Get the descriptions for all hotels
def get_hotel_description(browser, url):
    """
    Visit hotel page and extract description

    Args:
        browser: Selenium webdriver instance
        url: Hotel page URL
    Returns:
        str: Hotel description or None if not found
    """
    try:
        # Navigate to hotel page
        browser.get(url)
        time.sleep(3)  # Wait for page to load

        # Wait for and find description element
        description_element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//p[@data-testid="property-description"]'))
        )

        return description_element.text

    except Exception as e:
        print(f"Error getting description: {str(e)}")
        return None

# Create a new column for descriptions
df['description'] = None

# Iterate through each hotel in the dataframe
for index, row in df.iterrows():
    if row['link'] is not None:
        print(f"Processing hotel {index + 1}/{len(df)}: {row['name']}")

        # Get description
        description = get_hotel_description(browser, row['link'])

        # Update dataframe
        df.at[index, 'description'] = description

        # Optional: Add a small delay between requests to avoid overwhelming the server
        time.sleep(2)
    else:
        print(f"Skipping hotel {index + 1} - No link available")


In [ ]:
# Filter rows that have no description or empty string
missing_desc_rows = df[
    (df['description'].isnull()) | (df['description'].str.strip() == '')
]

# Re-run the extraction for these missing rows
for index, row in missing_desc_rows.iterrows():
    if row['link']:
        print(f"Re-processing hotel {index + 1}/{len(df)}: {row['name']}")
        description = get_hotel_description(browser, row['link'])
        df.at[index, 'description'] = description
        time.sleep(2)
    else:
        print(f"Skipping hotel {index + 1} - No link available")

In [ ]:
# Create check in day and check out day
df['check_in'] = pd.to_datetime('2025-05-16')
df['check_out'] = pd.to_datetime('2025-05-18')

In [ ]:
# Save the data
df.to_excel('barcelona_16_18.xlsx', index=False)